In [7]:
import pandas as pd
import glob
import os

In [8]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)                                


In [13]:


# datapath = "../data/WGS/Vilborg2015/wgs_stranded"
# outpath = "../data/WGS/Vilborg2015/wgs_stranded/normalized"
# makeFolders([outpath])
# vilborg bowtie2
f1 = "../data/WGS/Vilborg2015/Vilborg_LibrarySizes.tsv"
datapath = "../data/WGS/Vilborg2015/wgs_stranded_bowtie2"
outpath = f"{datapath}/normalized"

# NINA DOGS
# datapath = "/Users/m/Google_Drive/Scripts/2020/Dogcatcher2/data/WGS/NINA_DOGS/proseq"
# outpath = f"{datapath}/normalized"
# f1 = f"{datapath}/mapstats/WT_CTL_AS_LibrarySizes.tsv"




makeFolders([outpath])
df = pd.read_csv(f1,sep="\t")
df["minimum"] = df["mapped_reads"].min()
df["LibraryRatio"] = df["mapped_reads"] / df["minimum"]
df["normalized"] = df["mapped_reads"] / df["LibraryRatio"]
df

,sample,condition,mapped_reads,minimum,LibraryRatio,normalized
0,CTL2,C,76871547,76871547,1.000000,76871547.0
1,KCL2,T,89390920,76871547,1.162861,76871547.0
2,KCL3,T,125935282,76871547,1.638256,76871547.0
3,CTL3,C,99855029,76871547,1.298986,76871547.0
4,CTL1,C,107426818,76871547,1.397485,76871547.0
5,KCL1,T,90596764,76871547,1.178547,76871547.0


In [14]:
# df_data = pd.read_csv(f"{datapath}/{f1}",sep="\t",names=["chrom","start","stop","count"],
#                       dtype={"chrom":str,"start":float,"stop":float,"count":float})
# df_data.head

In [15]:
for strand in ["plu","min"]:
    for condition in df["condition"].unique():
        df_condition = df[df["condition"]==condition]
        num_in_conditions = len(df_condition)
        # Get first sample to make dataframe
        sample = df_condition["sample"].iloc[0]    
        f = glob.glob(f"{datapath}/{sample}_{strand}.wgs.regions.bed")[0]        
        df_final = pd.read_csv(f,sep="\t",names=["chrom","start","stop","count"],
        dtype={"chrom":str,"start":float,"stop":float,"count":float})
        del df_final["count"]
        for row in range(len(df_condition)):
            sample = df_condition["sample"].iloc[row]
            LibraryRatio = df_condition["LibraryRatio"].iloc[row]
            f = glob.glob(f"{datapath}/{sample}_{strand}.wgs.regions.bed")[0]
            print(f"Working on {f}")
            df_data = pd.read_csv(f,sep="\t",names=["chrom","start","stop","count"],
                      dtype={"chrom":str,"start":float,"stop":float,"count":float})
            df_final[f"normalized_{row}"] = df_data["count"] / LibraryRatio
        # Now average all samples    
        norm_list = matching = [s for s in list(df_final) if "normalized" in s]
        df_final["average"] = df_final[norm_list].sum(axis=1) / num_in_conditions
        df_final["start"] = df_final["start"].astype(int)
        df_final["stop"] = df_final["stop"].astype(int)
        df_final = df_final[["chrom","start","stop","average"]]
        fout = f"{outpath}/{condition}_{strand}.bed"
        print(f"Writing out {fout}")
        df_final.to_csv(fout,sep="\t",header=None,index=None)

Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL2_plu.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL3_plu.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL1_plu.wgs.regions.bed
Writing out ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/normalized/C_plu.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/KCL2_plu.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/KCL3_plu.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/KCL1_plu.wgs.regions.bed
Writing out ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/normalized/T_plu.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL2_min.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL3_min.wgs.regions.bed
Working on ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/CTL1_min.wgs.regions.bed
Writing out ../data/WGS/Vilborg2015/wgs_stranded_bowtie2/normalized/C_min.bed
Working on ../data/WGS/Vilborg2015/wg

In [ ]:
# df_final.head()